# install packages

In [1]:
!pip install pyyaml==5.3.1

  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.


In [2]:
!pip install gen3

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.
botocore 1.24.32 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.0.7 which is incompatible.


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Created wheel for cdislogging: filename=cdislogging-1.1.1-py3-none-any.whl size=7193 sha256=665701992ea56fb9ad7784799f65082bcae7e02ede28d7483a12b62d955c163a
  Stored in directory: c:\users\jdfuh\appdata\local\pip\cache\wheels\d0\23\c9\4598cfc85bdc3d639c83651918c39c3b369ad923ea58aa09c3
  Created wheel for drsclient: filename=drsclient-0.2.3-py3-none-any.whl size=7424 sha256=b5292e4afc92432339debb109f0e34a4131b5fd7a262c2844697fa2ed8ec59d8
  Stored in directory: c:\users\jdfuh\appdata\local\pip\cache\wheels\3c\be\76\65b86ab9faf71213aab39192969cfccd1b05291b7f9028cc8b
  Created wheel for dictionaryutils: filename=dictionaryutil

In [3]:
!pip install pydicom

In [4]:
import gen3
import os
import time

In [5]:
# test pulling MIDRC GUIDs -- this works
# os.system("gen3 --auth midrc_credentials.json --endpoint data.midrc.org drs-pull object dg.MD1R/ea669b5e-ae51-40ba-b375-ed23a9cd1855")
          

# write nextflow config

In [6]:
%%writefile nextflow.config

process {
    withLabel: download_data {
        executor = 'local'
    }
    withLabel: publish_data {
        executor = 'local'
    }
    withLabel: dcm2png {
        executor = 'local'
    }
}

workDir = 'sdk_data'

Writing nextflow.config


# Pull some midrc guids to test

In [8]:
%%writefile download_data.py

import gen3
import os

output_dir = 'data'

guids = ['dg.MD1R/ea669b5e-ae51-40ba-b375-ed23a9cd1855',
         'dg.MD1R/a745ed98-0cb9-4537-826b-13b2e354e8bb',
         'dg.MD1R/e604979a-c71b-4ec6-b8a0-959837b86384',
         'dg.MD1R/b5cee98d-46ff-4438-aa00-90727a383340',
         'dg.MD1R/8a5a5579-7925-432d-a614-3ed208f1c182',
         'dg.MD1R/33034812-47f3-4c0e-b60b-fa7a2a04ecda',
         'dg.MD1R/5ca987c5-c660-4785-a67d-a3424cc8ec6e',
         'dg.MD1R/44148117-1858-49ef-b30f-d239abfaff80',
         'dg.MD1R/9ea205e8-a774-4318-a323-95eadda9bc5c',
         'dg.MD1R/09ece36f-a0fa-48e8-8fc2-62110eaae570']

midrc_creds_file = "C:\Users\jdfuh\source\repos\ContainerizationTest\midrc_credentials.json"
endpoint = 'data.midrc.org'

for guid in guids:
    print('pulling guid {}'.format(guid))
    os.system("gen3 --auth {} --endpoint {} drs-pull object {}".format(midrc_creds_file, endpoint, guid))

Overwriting download_data.py


In [9]:
%%writefile main.nf
#!/usr/bin/env nextflow

process DownloadData {
    
    label 'download_data'
    
    tag 'initial_task'
    
    output:
    stdout emit: download_data_log
    path('**/*.dcm'), emit: dicom_image
    
    script:
    """
    python3 ${baseDir}/download_data.py
    """
}

process PublishData {
    
    label 'publish_data'
    
    publishDir "${baseDir}/dicom_files", mode: 'copy'
    
    input:
    path(dicom_image) 
    
    output:
    stdout emit: convert_to_png_log
    
    script:
    """
    echo $dicom_image
    """
}

// Define the entry workflow (initial workflow for Nextflow to run)
workflow {
    DownloadData()
    PublishData(DownloadData.out.dicom_image)
}


Writing main.nf


In [10]:
!nextflow run main.nf -dsl2

'nextflow' is not recognized as an internal or external command,
operable program or batch file.
